# Title Here

### Connect to your workspace

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [8]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION = "5bab2ebd-b16a-469d-8aa4-abdaea7f9e17"
RESOURCE_GROUP = "rg-dp100-l"
WS_NAME = "mlw-dp100-l"
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

Create Data store and upload training data

In [9]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "../data"
dataset_name = "diabetes-data-train"

patient_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for diabetes classification",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset_unlabeled)

Data({'path': 'azureml://subscriptions/5bab2ebd-b16a-469d-8aa4-abdaea7f9e17/resourcegroups/rg-dp100-l/workspaces/mlw-dp100-l/datastores/workspaceblobstore/paths/LocalUpload/177b0cdcf5bcd30ff675ca95a93cbccd/data/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'diabetes-data-train', 'description': 'An unlabeled dataset for diabetes classification', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/5bab2ebd-b16a-469d-8aa4-abdaea7f9e17/resourceGroups/rg-dp100-l/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-l/data/diabetes-data-train/versions/6', 'Resource__source_path': '', 'base_path': 'c:\\Users\\shenglinxu\\OneDrive - Microsoft\\projects\\personal_projects\\MLOps-basics\\notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x0000024ADDEA6C50>, 'serialize': <msre

In [10]:
patient_dataset_unlabeled = ml_client.data.get(
    name="diabetes-data-train", label="latest"
)

print(patient_dataset_unlabeled)

creation_context:
  created_at: '2024-11-28T23:11:51.946346+00:00'
  created_by: System Administrator
  created_by_type: User
  last_modified_at: '2024-11-28T23:11:51.960205+00:00'
description: An unlabeled dataset for diabetes classification
id: /subscriptions/5bab2ebd-b16a-469d-8aa4-abdaea7f9e17/resourceGroups/rg-dp100-l/providers/Microsoft.MachineLearningServices/workspaces/mlw-dp100-l/data/diabetes-data-train/versions/6
name: diabetes-data-train
path: azureml://subscriptions/5bab2ebd-b16a-469d-8aa4-abdaea7f9e17/resourcegroups/rg-dp100-l/workspaces/mlw-dp100-l/datastores/workspaceblobstore/paths/LocalUpload/177b0cdcf5bcd30ff675ca95a93cbccd/data/
properties: {}
tags: {}
type: uri_folder
version: '6'



Load components

In [16]:
from azure.ai.ml import load_component
parent_dir = "../components/"

fix_missing_data = load_component(source=parent_dir + "fix-missing-data.yml")
normalize_data = load_component(source=parent_dir + "normalize-data.yml")
train_decision_tree = load_component(source=parent_dir + "train-decision-tree.yml")
train_logistic_regression = load_component(source=parent_dir + "train-logistic-regression.yml")




Build pipeline

In [17]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def diabetes_classification(pipeline_job_input):
    clean_data = fix_missing_data(input_data=pipeline_job_input)
    normalized_data = normalize_data(input_data=clean_data.outputs.output_data)
    train_model_decision_tree = train_decision_tree(training_data=normalized_data.outputs.output_data)
    train_model_logistic_regression = train_logistic_regression(training_data=normalized_data.outputs.output_data)


    return {
        "pipeline_job_transformed_data": normalized_data.outputs.output_data,
        "pipeline_job_trained_model_decision_tree": train_model_decision_tree.outputs.model_output_decision_tree,
        "pipeline_job_trained_model_logistic_regression": train_model_logistic_regression.outputs.model_output_logistic_reg,
    }

pipeline_job = diabetes_classification(Input(type=AssetTypes.URI_FILE, path=patient_dataset_unlabeled.path))
print(pipeline_job)

display_name: diabetes_classification
type: pipeline
inputs:
  pipeline_job_input:
    type: uri_file
    path: azureml://subscriptions/5bab2ebd-b16a-469d-8aa4-abdaea7f9e17/resourcegroups/rg-dp100-l/workspaces/mlw-dp100-l/datastores/workspaceblobstore/paths/LocalUpload/177b0cdcf5bcd30ff675ca95a93cbccd/data/
outputs:
  pipeline_job_transformed_data:
    type: uri_folder
  pipeline_job_trained_model_decision_tree:
    type: uri_folder
  pipeline_job_trained_model_logistic_regression:
    type: uri_folder
jobs:
  clean_data:
    type: command
    inputs:
      input_data:
        path: ${{parent.inputs.pipeline_job_input}}
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: remove_empty_rows
      version: '1'
      display_name: Remove Empty Rows
      type: command
      inputs:
        input_data:
          type: uri_folder
      outputs:
        output_data:
          type: uri_folder
      command: python fix-missing-data

In [18]:

# set pipeline level compute
pipeline_job.settings.default_compute = "aml-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# print the pipeline job again to review the changes
print(pipeline_job)

display_name: diabetes_classification
type: pipeline
inputs:
  pipeline_job_input:
    type: uri_file
    path: azureml://subscriptions/5bab2ebd-b16a-469d-8aa4-abdaea7f9e17/resourcegroups/rg-dp100-l/workspaces/mlw-dp100-l/datastores/workspaceblobstore/paths/LocalUpload/177b0cdcf5bcd30ff675ca95a93cbccd/data/
outputs:
  pipeline_job_transformed_data:
    type: uri_folder
  pipeline_job_trained_model_decision_tree:
    type: uri_folder
  pipeline_job_trained_model_logistic_regression:
    type: uri_folder
jobs:
  clean_data:
    type: command
    inputs:
      input_data:
        path: ${{parent.inputs.pipeline_job_input}}
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: remove_empty_rows
      version: '1'
      display_name: Remove Empty Rows
      type: command
      inputs:
        input_data:
          type: uri_folder
      outputs:
        output_data:
          type: uri_folder
      command: python fix-missing-data

In [19]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_diabetes_training"
)
pipeline_job

Uploading src (0.01 MBs): 100%|##########| 8451/8451 [00:00<00:00, 53858.43it/s]


pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Experiment,Name,Type,Status,Details Page
pipeline_diabetes_training,shy_hominy_42m2v1djkv,pipeline,NotStarted,Link to Azure Machine Learning studio


In [24]:
# List all child jobs in the job
# List all child jobs in the job
child_jobs = ml_client.jobs.list(parent_job_name=pipeline_job.name)

# Traverse and download all the outputs of child job
for child_job in child_jobs:
    ml_client.jobs.download(name=child_job.name, output_name="model_output_decision_tree", download_path="../outputs")

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

model_name = 'diabetes-decision-tree-model'
model = ml_client.models.create_or_update(
    Model(name=model_name, path='azureml:azureml_4cfaf918-27b0-42e6-98e3-3c99270db33b_output_data_model_output:1', type=AssetTypes.MLFLOW_MODEL)
)